In [68]:
!pip install scikit-fuzzy
!pip install EasyGA

We first import necessary libraries

In [69]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import FunctionTransformer
import pandas as pd
import EasyGA
import os
import random



setup_fuzzy_system_optimized returns the control system simulation variables for the genetic fuzzy tree for an input chromosome



In [70]:
def setup_fuzzy_system_optimized(chromosome):
  vals = chromosome.gene_value_list[0]


  l = [0,0,vals[0]]
  m = [0,vals[1],10]
  h = [vals[2],vals[2],10]

    # Define input and output variables for Simulation 1: Determine food quality
  temperature = ctrl.Antecedent(np.linspace(0, 10, 11), 'temperature')
  flavor = ctrl.Antecedent(np.linspace(0, 10, 11), 'flavor')
  portion_size = ctrl.Antecedent(np.linspace(0, 10, 11), 'portion_size')
  food_quality = ctrl.Consequent(np.linspace(0, 10, 11), 'food_quality')

  # Manually define membership functions for 'flavor'
  flavor['poor'] = fuzz.trimf(flavor.universe, l)
  flavor['average'] = fuzz.trimf(flavor.universe, m)
  flavor['good'] = fuzz.trimf(flavor.universe, h)

  # Manually define membership functions for 'temperature'
  temperature['poor'] = fuzz.trimf(temperature.universe, l)
  temperature['average'] = fuzz.trimf(temperature.universe, m)
  temperature['good'] = fuzz.trimf(temperature.universe, h)

  # Manually define membership functions for 'portion_size'
  portion_size['poor'] = fuzz.trimf(portion_size.universe, l)
  portion_size['average'] = fuzz.trimf(portion_size.universe, m)
  portion_size['good'] = fuzz.trimf(portion_size.universe, h)

  # Manually define membership functions for 'food_quality'
  food_quality['poor'] = fuzz.trimf(food_quality.universe, l)
  food_quality['average'] = fuzz.trimf(food_quality.universe, m)
  food_quality['good'] = fuzz.trimf(food_quality.universe, h)

  food_rules = [
      ctrl.Rule(temperature['poor'] & flavor['poor'] & portion_size['poor'], food_quality['poor']),
      ctrl.Rule(temperature['poor'] & flavor['poor'] & portion_size['average'], food_quality['poor']),
      ctrl.Rule(temperature['poor'] & flavor['poor'] & portion_size['good'], food_quality['poor']),
      ctrl.Rule(temperature['poor'] & flavor['average'] & portion_size['poor'], food_quality['poor']),
      ctrl.Rule(temperature['poor'] & flavor['average'] & portion_size['average'], food_quality['average']),
      ctrl.Rule(temperature['poor'] & flavor['average'] & portion_size['good'], food_quality['average']),
      ctrl.Rule(temperature['poor'] & flavor['good'] & portion_size['poor'], food_quality['average']),
      ctrl.Rule(temperature['poor'] & flavor['good'] & portion_size['average'], food_quality['average']),
      ctrl.Rule(temperature['poor'] & flavor['good'] & portion_size['good'], food_quality['good']),

      ctrl.Rule(temperature['average'] & flavor['poor'] & portion_size['poor'], food_quality['poor']),
      ctrl.Rule(temperature['average'] & flavor['poor'] & portion_size['average'], food_quality['average']),
      ctrl.Rule(temperature['average'] & flavor['poor'] & portion_size['good'], food_quality['average']),
      ctrl.Rule(temperature['average'] & flavor['average'] & portion_size['poor'], food_quality['average']),
      ctrl.Rule(temperature['average'] & flavor['average'] & portion_size['average'], food_quality['average']),
      ctrl.Rule(temperature['average'] & flavor['average'] & portion_size['good'], food_quality['average']),
      ctrl.Rule(temperature['average'] & flavor['good'] & portion_size['poor'], food_quality['average']),
      ctrl.Rule(temperature['average'] & flavor['good'] & portion_size['average'], food_quality['average']),
      ctrl.Rule(temperature['average'] & flavor['good'] & portion_size['good'], food_quality['good']),

      ctrl.Rule(temperature['good'] & flavor['poor'] & portion_size['poor'], food_quality['poor']),
      ctrl.Rule(temperature['good'] & flavor['poor'] & portion_size['average'], food_quality['average']),
      ctrl.Rule(temperature['good'] & flavor['poor'] & portion_size['good'], food_quality['good']),
      ctrl.Rule(temperature['good'] & flavor['average'] & portion_size['poor'], food_quality['average']),
      ctrl.Rule(temperature['good'] & flavor['average'] & portion_size['average'], food_quality['good']),
      ctrl.Rule(temperature['good'] & flavor['average'] & portion_size['good'], food_quality['good']),
      ctrl.Rule(temperature['good'] & flavor['good'] & portion_size['poor'], food_quality['good']),
      ctrl.Rule(temperature['good'] & flavor['good'] & portion_size['average'], food_quality['good']),
      ctrl.Rule(temperature['good'] & flavor['good'] & portion_size['good'], food_quality['good'])
  ]

  food_ctrl = ctrl.ControlSystem(food_rules)

  # Define input and output variables
  attentiveness = ctrl.Antecedent(np.linspace(0, 10, 11), 'attentiveness')
  friendliness = ctrl.Antecedent(np.linspace(0, 10, 11), 'friendliness')
  speed = ctrl.Antecedent(np.linspace(0, 10, 11), 'speed')
  service_quality = ctrl.Consequent(np.linspace(0, 10, 11), 'service_quality')

  # Manually define membership functions for 'attentiveness'
  attentiveness['poor'] = fuzz.trimf(attentiveness.universe, l)
  attentiveness['average'] = fuzz.trimf(attentiveness.universe, m)
  attentiveness['good'] = fuzz.trimf(attentiveness.universe, h)

  # Manually define membership functions for 'friendliness'
  friendliness['poor'] = fuzz.trimf(friendliness.universe, l)
  friendliness['average'] = fuzz.trimf(friendliness.universe, m)
  friendliness['good'] = fuzz.trimf(friendliness.universe, h)

  # Manually define membership functions for 'speed'
  speed['poor'] = fuzz.trimf(speed.universe, l)
  speed['average'] = fuzz.trimf(speed.universe, m)
  speed['good'] = fuzz.trimf(speed.universe, h)

  service_quality['poor'] = fuzz.trimf(service_quality.universe, l)
  service_quality['average'] = fuzz.trimf(service_quality.universe, m)
  service_quality['good'] = fuzz.trimf(service_quality.universe, h)

  service_rules = [
      ctrl.Rule(attentiveness['poor'] & friendliness['poor'] & speed['poor'], service_quality['poor']),
      ctrl.Rule(attentiveness['poor'] & friendliness['poor'] & speed['average'], service_quality['poor']),
      ctrl.Rule(attentiveness['poor'] & friendliness['poor'] & speed['good'], service_quality['poor']),
      ctrl.Rule(attentiveness['poor'] & friendliness['average'] & speed['poor'], service_quality['poor']),
      ctrl.Rule(attentiveness['poor'] & friendliness['average'] & speed['average'], service_quality['average']),
      ctrl.Rule(attentiveness['poor'] & friendliness['average'] & speed['good'], service_quality['average']),
      ctrl.Rule(attentiveness['poor'] & friendliness['good'] & speed['poor'], service_quality['average']),
      ctrl.Rule(attentiveness['poor'] & friendliness['good'] & speed['average'], service_quality['average']),
      ctrl.Rule(attentiveness['poor'] & friendliness['good'] & speed['good'], service_quality['good']),

      ctrl.Rule(attentiveness['average'] & friendliness['poor'] & speed['poor'], service_quality['poor']),
      ctrl.Rule(attentiveness['average'] & friendliness['poor'] & speed['average'], service_quality['average']),
      ctrl.Rule(attentiveness['average'] & friendliness['poor'] & speed['good'], service_quality['average']),
      ctrl.Rule(attentiveness['average'] & friendliness['average'] & speed['poor'], service_quality['average']),
      ctrl.Rule(attentiveness['average'] & friendliness['average'] & speed['average'], service_quality['average']),
      ctrl.Rule(attentiveness['average'] & friendliness['average'] & speed['good'], service_quality['average']),
      ctrl.Rule(attentiveness['average'] & friendliness['good'] & speed['poor'], service_quality['average']),
      ctrl.Rule(attentiveness['average'] & friendliness['good'] & speed['average'], service_quality['average']),
      ctrl.Rule(attentiveness['average'] & friendliness['good'] & speed['good'], service_quality['good']),

      ctrl.Rule(attentiveness['good'] & friendliness['poor'] & speed['poor'], service_quality['poor']),
      ctrl.Rule(attentiveness['good'] & friendliness['poor'] & speed['average'], service_quality['average']),
      ctrl.Rule(attentiveness['good'] & friendliness['poor'] & speed['good'], service_quality['good']),
      ctrl.Rule(attentiveness['good'] & friendliness['average'] & speed['poor'], service_quality['average']),
      ctrl.Rule(attentiveness['good'] & friendliness['average'] & speed['average'], service_quality['good']),
      ctrl.Rule(attentiveness['good'] & friendliness['average'] & speed['good'], service_quality['good']),
      ctrl.Rule(attentiveness['good'] & friendliness['good'] & speed['poor'], service_quality['good']),
      ctrl.Rule(attentiveness['good'] & friendliness['good'] & speed['average'], service_quality['good']),
      ctrl.Rule(attentiveness['good'] & friendliness['good'] & speed['good'], service_quality['good'])
  ]


  # Create a control system
  service_quality_ctrl = ctrl.ControlSystem(service_rules)


  lt =   [i * 3 for i in l]

  mt =   [i * 3 for i in m]

  ht =   [i * 3 for i in h]


  # Define input and output variables for tipping
  food_quality_antecedent = ctrl.Antecedent(np.linspace(0, 10, 11), 'food_quality')
  service_quality_antecedent = ctrl.Antecedent(np.linspace(0, 10, 11), 'service_quality')

  # Manually define membership functions for 'food_quality_antecedent'
  food_quality_antecedent['poor'] = fuzz.trimf(food_quality_antecedent.universe, l)
  food_quality_antecedent['average'] = fuzz.trimf(food_quality_antecedent.universe, m)
  food_quality_antecedent['good'] = fuzz.trimf(food_quality_antecedent.universe, h)

  # Manually define membership functions for 'service_quality_antecedent'
  service_quality_antecedent['poor'] = fuzz.trimf(service_quality_antecedent.universe, l)
  service_quality_antecedent['average'] = fuzz.trimf(service_quality_antecedent.universe, m)
  service_quality_antecedent['good'] = fuzz.trimf(service_quality_antecedent.universe, h)

  # Define input and output variables for tipping
  tip = ctrl.Consequent(np.linspace(0, 30, 31), 'tip')

  tip['poor'] = fuzz.trimf(tip.universe,lt)  # Example: Low tip ranges from 0% to 10%
  tip['average'] = fuzz.trimf(tip.universe, mt)  # Example: Medium tip ranges from 10% to 20%
  tip['good'] = fuzz.trimf(tip.universe, ht)  # Example: High tip ranges from 20% to 30%

  tip_rules = [
      ctrl.Rule(food_quality_antecedent['poor'] & service_quality_antecedent['poor'], tip['poor']),
      ctrl.Rule(food_quality_antecedent['poor'] & service_quality_antecedent['average'], tip['poor']),
      ctrl.Rule(food_quality_antecedent['poor'] & service_quality_antecedent['good'], tip['average']),

      ctrl.Rule(food_quality_antecedent['average'] & service_quality_antecedent['poor'], tip['poor']),
      ctrl.Rule(food_quality_antecedent['average'] & service_quality_antecedent['average'], tip['average']),
      ctrl.Rule(food_quality_antecedent['average'] & service_quality_antecedent['good'], tip['good']),

      ctrl.Rule(food_quality_antecedent['good'] & service_quality_antecedent['poor'], tip['average']),
      ctrl.Rule(food_quality_antecedent['good'] & service_quality_antecedent['average'], tip['good']),
      ctrl.Rule(food_quality_antecedent['good'] & service_quality_antecedent['good'], tip['good'])
      ]

# Place all rules in a list called tip_rules

  tipping_ctrl = ctrl.ControlSystem(tip_rules)

  food_q = ctrl.ControlSystemSimulation(food_ctrl)
  service_q = ctrl.ControlSystemSimulation(service_quality_ctrl)
  tipping_system = ctrl.ControlSystemSimulation(tipping_ctrl)

  return food_q,service_q,tipping_system





setup_fuzzy_system_unoptimized returns the control system simulation variables for the unoptimized fuzzy tree from lab 4 (same rules etc)

In [71]:
def setup_fuzzy_system_unoptimizied():

# Define input and output variables for Simulation 1: Determine food quality
  temperature = ctrl.Antecedent(np.linspace(0, 10, 11), 'temperature')
  flavor = ctrl.Antecedent(np.linspace(0, 10, 11), 'flavor')
  portion_size = ctrl.Antecedent(np.linspace(0, 10, 11), 'portion_size')
  food_quality = ctrl.Consequent(np.linspace(0, 10, 11), 'food_quality')

  # Auto-generate membership functions for input variables
  flavor.automf(3,names = ['bland','average','delicious'])
  temperature.automf(3,names = ['cold','medium','hot'])
  portion_size.automf(3, names = ['small', 'medium', 'large'])

  # Auto-generate membership functions for output
  food_quality.automf(3,names = ['poor','average','good'])

  # Rules for Food Quality "good"
  rule1_food = ctrl.Rule((temperature['medium'] | temperature['hot']) & flavor['delicious'] & (portion_size['medium'] | portion_size['large']), food_quality['good'])

  # Rules for Food Quality "average"
  rule2_food = ctrl.Rule((temperature['cold'] | temperature['medium']|temperature['hot']) & flavor['average'] & (portion_size['small'] | portion_size['medium'] | portion_size['large']), food_quality['average'])
  rule3_food = ctrl.Rule((temperature['cold']) & flavor['delicious'] & (portion_size['small']|portion_size['medium']|portion_size['large']), food_quality['average'])
  rule4_food = ctrl.Rule(temperature['hot'] & flavor['bland'] & portion_size['large'], food_quality['average'])
  rule5_food = ctrl.Rule(temperature['hot'] & flavor['delicious'] & portion_size['small'], food_quality['average'])
  rule6_food = ctrl.Rule(temperature['medium'] & flavor['delicious'] & portion_size['small'], food_quality['average'])

  # Rules for Food Quality "poor"
  rule7_food = ctrl.Rule((temperature['cold'] | temperature['medium']) & flavor['bland'] & (portion_size['small'] | portion_size['medium'] | portion_size['large']), food_quality['poor'])
  rule8_food = ctrl.Rule(temperature['hot'] & flavor['bland'] & (portion_size['small']|portion_size['medium']), food_quality['poor'])


  food_ctrl = ctrl.ControlSystem([ rule1_food, rule2_food,rule3_food,rule4_food, rule5_food,rule6_food,rule7_food, rule8_food])

  # Define input and output variables
  attentiveness = ctrl.Antecedent(np.linspace(0, 10, 11), 'attentiveness')
  friendliness = ctrl.Antecedent(np.linspace(0, 10, 11), 'friendliness')
  speed = ctrl.Antecedent(np.linspace(0, 10, 11), 'speed')
  service_quality = ctrl.Consequent(np.linspace(0, 10, 11), 'service_quality')

  # Auto-generate membership functions for input variables
  attentiveness.automf(3, names=['low', 'average', 'high'])
  friendliness.automf(3, names=['low', 'average', 'high'])
  speed.automf(3, names=['slow', 'average', 'fast'])
  # Manually define membership functions for 'speed'
  speed['slow'] = fuzz.trimf(speed.universe, [0,0,5])
  speed['average'] = fuzz.trimf(speed.universe, [0,5,10])
  speed['fast'] = fuzz.trimf(speed.universe, [5,10,10])

  # Auto-generate membership functions for output
  service_quality.automf(3,names = ['poor','average','good'])

  # Generate rules
  rule1_service = ctrl.Rule((attentiveness['low']|attentiveness['average']) & (friendliness['low']) & (speed['slow']|speed['average']), service_quality['poor'])
  rule2_service = ctrl.Rule(attentiveness['low'] & (friendliness['average']) & speed['slow'], service_quality['poor'])
  rule3_service = ctrl.Rule((attentiveness['high']) & (friendliness['average']|friendliness['low'])& speed['slow'], service_quality['poor'])
  rule4_service = ctrl.Rule((attentiveness['low']) & friendliness['high']& speed['slow'], service_quality['poor'])

  rule5_service = ctrl.Rule(attentiveness['average'] & friendliness['average'] & (speed['slow']|speed['average']), service_quality['average'])
  rule6_service = ctrl.Rule(attentiveness['average'] & friendliness['high'] & speed['slow'], service_quality['average'])
  rule7_service = ctrl.Rule((attentiveness['high']) & friendliness['low']& (speed['fast']|speed['average']), service_quality['average'])
  rule8_service = ctrl.Rule((attentiveness['low']) & (friendliness['high']|friendliness['average'])& (speed['average']|speed['fast']), service_quality['average'])
  rule9_service = ctrl.Rule((attentiveness['low']|attentiveness['average']) & friendliness['low']& speed['fast'], service_quality['average'])

  rule10_service = ctrl.Rule((attentiveness['average']|attentiveness['high']) & friendliness['high']& (speed['average']|speed['fast']), service_quality['good'])
  rule11_service = ctrl.Rule((attentiveness['high']) & friendliness['average']& (speed['fast']|speed['average']), service_quality['good'])
  rule12_service = ctrl.Rule((attentiveness['average']) & friendliness['average']& speed['fast'], service_quality['good'])
  rule13_service = ctrl.Rule(attentiveness['high'] & friendliness['high'] & speed['slow'], service_quality['good'])

  # Create a control system
  service_quality_ctrl = ctrl.ControlSystem([rule1_service, rule2_service, rule3_service,rule4_service, rule5_service,rule6_service, rule7_service,rule8_service,rule9_service,rule10_service,rule11_service,rule12_service,rule13_service])

    # Define input and output variables for tipping
  food_quality_antecedent = ctrl.Antecedent(np.linspace(0, 10, 11), 'food_quality')
  service_quality_antecedent = ctrl.Antecedent(np.linspace(0, 10, 11), 'service_quality')

  # Define linguistic terms for food_quality and service_quality
  food_quality_antecedent.automf(3, names=['poor', 'average', 'good'])
  service_quality_antecedent.automf(3, names=['poor', 'average', 'good'])

  # Define input and output variables for tipping
  tip = ctrl.Consequent(np.linspace(0, 30, 31), 'tip')

  # Define linguistic terms for tip
  tip['low'] = fuzz.trimf(tip.universe, [0, 5, 10])  # Example: Low tip ranges from 0% to 10%
  tip['medium'] = fuzz.trimf(tip.universe, [10, 15, 20])  # Example: Medium tip ranges from 10% to 20%
  tip['high'] = fuzz.trimf(tip.universe, [20, 25, 30])  # Example: High tip ranges from 20% to 30%

  # Create the tipping control system based on the outputs of the two simulations

  rule1_tip = ctrl.Rule(food_quality_antecedent['poor'] | service_quality_antecedent['poor'], tip['low'])
  rule2_tip = ctrl.Rule(food_quality_antecedent['average'] & service_quality_antecedent['average'], tip['medium'])
  rule3_tip = ctrl.Rule(food_quality_antecedent['average'] & service_quality_antecedent['good'], tip['high'])
  rule4_tip = ctrl.Rule(food_quality_antecedent['good'] & service_quality_antecedent['average'], tip['high'])
  rule5_tip = ctrl.Rule(food_quality_antecedent['good'] & service_quality_antecedent['good'], tip['high'])

  tipping_ctrl = ctrl.ControlSystem([rule1_tip, rule2_tip, rule3_tip, rule4_tip, rule5_tip])

  food_q = ctrl.ControlSystemSimulation(food_ctrl)
  service_q = ctrl.ControlSystemSimulation(service_quality_ctrl)
  tipping_system = ctrl.ControlSystemSimulation(tipping_ctrl)

  return food_q,service_q,tipping_system

We encapsulate our control simulation variables in a FuzzyLogicTransformer class. This is also needed for sklearn pipeline method.

Since no training occurs, the fit method is empty. Instead we use the transform method with takes input as a dictionary mapping inputs to the corresponding values.

This data dictionary first passes through the clean_data method where we clip value to be between 0,1 and then scale to 0,10.

In transform, we redirect user input to the first stage FISMs for food quality and service quality. The outputs of these 2 are then redirected to the final tipping FISM.

In [72]:

def clean_data(X):
    X['food temperature'] = max(0, min(1, X.get('food temperature', 0))) * 10
    X['food flavor'] = max(0, min(1, X.get('food flavor', 0))) * 10
    X['portion size'] = max(0, min(1, X.get('portion size', 0))) * 10
    X['attentiveness'] = max(0, min(1, X.get('attentiveness', 0))) * 10
    X['friendliness'] = max(0, min(1, X.get('friendliness', 0))) * 10
    X['speed of service'] = max(0, min(1, X.get('speed of service', 0))) * 10

    for key in X:
        if X[key] == 0:
            X[key] = 0.0001
        elif X[key] == 10:
            X[key] = 9.9999

    return X


class FuzzyLogicTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, food_q, service_q, tipping_system):
        self.food_q = food_q
        self.service_q = service_q
        self.tipping_system = tipping_system

    def fit(self, X, y=None):
        return self

    def transform(self, X):

        self.food_q.input['temperature'] = float(X['food temperature'])
        self.food_q.input['flavor'] = float(X['food flavor'])
        self.food_q.input['portion_size'] = float(X['portion size'])

        self.service_q.input['attentiveness'] = float(X['attentiveness'])
        self.service_q.input['friendliness'] = float(X['friendliness'])
        self.service_q.input['speed'] = float(X['speed of service'])

        self.food_q.compute()
        self.service_q.compute()

        self.tipping_system.input['food_quality'] = self.food_q.output['food_quality']
        self.tipping_system.input['service_quality'] = self.service_q.output['service_quality']
        self.tipping_system.compute()

        tip_amount = self.tipping_system.output['tip']

        return tip_amount




We load the train and test data into pandas datframes. This data is available globally.

We define the fitness method with optional arguments chromosome, test, mode.
chromosome is generated by the EasyGA library and is used when the mode is set to "op" for optimized.

This is for best chromosome exploration.

If chromosome is None and mode is "unop" we are getting the error when the data set is run on the unoptimized genetic fuzzy tree from lab 4.

Data is loaded from each data frame row and passed to our pipeline object.

First step is data cleaning and transformation where the pipeline calls clean_data method.

The resultant data is then passed to the transform() method in the FuzzyLogicTransformer

fitness returns the total error (sum of errors per row for the predicted tip vs actual tip)

In [73]:
# load training and testing data here

global test_data
global train_data

test_data = pd.read_csv('/content/tipper_test.csv')
train_data = pd.read_csv('/content/tipper_train.csv')

def fitness(chromosome = None, test = None, mode = None):
  data = None

  if not test:
    data = train_data
  else:
    data = test_data

  if mode == "op":
    food_q, service_q, tipping_system = setup_fuzzy_system_optimized(chromosome)
  elif mode == "unop":
    food_q, service_q, tipping_system = setup_fuzzy_system_unoptimizied()

  pipeline = Pipeline([
      ('data_cleaning', FunctionTransformer(clean_data)),
      ('fuzzy_logic', FuzzyLogicTransformer(food_q, service_q, tipping_system))
  ])

  total_error = 0
  for index, row in data.iterrows():
    inputs = {
    'food temperature': row['food temperature'],
    'food flavor': row['food flavor'],
    'portion size': row['portion size'],
    'attentiveness': row['attentiveness'],
    'friendliness': row['friendliness'],
    'speed of service': row['speed of service']
    }
    actual_tip = row['tip']
    predicted_tip = pipeline.transform(inputs)
    error = abs(actual_tip - predicted_tip)
    total_error += error
  return total_error


generate_chromosome defines the constraints for the chromosome and returns a chromosome : list of three random ints from the ranges described [(0,5),(3,7),(5,10)]

get_best_chromosome takes the population size and generation goal parameters as inputs and gets the best chromosome by running the generate_chromosome and the fitness functions.

It gets the chromosome with the lowest error and returns it.

evaluate_genetic_fuzzy_tree takes the best chromosome as argument and gets the error on the test set for unoptimized fuzzy tree and the optimized genetic fuzzy tree (it uses the best_chromosome arguement)

We then print the results


In [ ]:
def generate_chromosome():

    l = np.random.randint(0,5)
    m = np.random.randint(3,7)
    h = np.random.randint(5,10)

    sorted_vals = [l,m,h]

    return sorted_vals

def get_best_chromosome(population_size, generation_goal):
  database_path = '/content/database.db'
  journal_path = '/content/database.db-journal'
  if os.path.exists(database_path):
        # If it exists, delete the file
      os.remove(database_path)
  if os.path.exists(journal_path):
        # If it exists, delete the file
    os.remove(journal_path)
  ga = EasyGA.GA()
  ga.gene_impl = lambda: generate_chromosome()
  ga.chromosome_length = 1
  ga.population_size = population_size
  ga.target_fitness_type = 'min'
  ga.generation_goal = generation_goal
  ga.fitness_function_impl = lambda chromosome: fitness(chromosome, test = False, mode = "op")
  ga.evolve()
  ga.print_best_chromosome()
  y = ga.population[0]
  return y

def evaluate_genetic_fuzzy_tree(best_chromosome):
  unoptimized_test_error = fitness(None,test = True, mode = "unop")
  print(f"Unoptimized Genetic Fuzzy Tree Error: {unoptimized_test_error}")
  optimized_test_error = fitness(best_chromosome, test = True, mode = "op")
  print(f"Optimized Genetic Fuzzy Tree Error: {optimized_test_error}")

best_chromosome = get_best_chromosome(10,20)
evaluate_genetic_fuzzy_tree(best_chromosome)
